* BINARIO DE IMAGEN

In [11]:
import cv2
import matplotlib.pyplot as plt

from src.binario import binarizar


img = cv2.imread("data/pruebas/a.jpg")
mask = binarizar(img)


cv2.imshow("Binaria",mask)
cv2.waitKey(0)
cv2.destroyAllWindows()

* MOSTRAR FEATURES DE CONTORNO

In [12]:
import pandas as pd

from src.features import extraer_features_desde_mask

# EXTRAER FEATURES (forma recomendada)
lista_features = extraer_features_desde_mask(
    mask,
    min_area_rel=0.001,
    min_hole_area_rel=0.0005
)

df = pd.DataFrame(lista_features)
df


,contour_index,area,perimeter,area_norm,perimeter_norm,aspect_ratio,extent,solidity,circularity,num_holes,hole_area_ratio,num_vertices_rdp,vertices_perimeter_ratio,hu1,hu2,hu3,hu4,hu5,hu6
0,0,6812.5,698.54119,0.025988,1.364338,3.407194,0.182401,0.597326,0.175442,0,0.0,7,0.010021,0.131741,0.302721,1.719903,1.983061,3.834556,2.134505


* MOSTRAR CONTORNOS FILTRADOS

In [13]:
from src.contornos import filtrar_contornos

img=cv2.resize(img, (512, 512), interpolation=cv2.INTER_NEAREST)

contornos, hierarchy = filtrar_contornos(
    mask,
    min_area_rel=0.001,
    borde_rel=0.95,
    mode=cv2.RETR_TREE
)

print(f"Contornos encontrados: {len(contornos)}")

# Dibujar los contornos en la imagen original
img_contornos = img.copy()

# color verde (B, G, R)
color = (0, 255, 0)
grosor = 2

# Dibujar todos los contornos filtrados
for cnt in contornos:
    cv2.drawContours(img_contornos, [cnt], -1, color, grosor)

cv2.imshow("Contornos filtrados", img_contornos)
cv2.waitKey(0)
cv2.destroyAllWindows()

Contornos encontrados: 1


* IDENTIFICAR PIEZAS CON K-MEANS

In [ ]:
from src.predecir import predecir

rutas = [
    "data/muestras/a.jpg",
    "data/muestras/f.jpg",
    "data/muestras/g.jpg",
    "data/muestras/h.jpg",
]

for r in rutas:
    pred, info = predecir(r)
    print(r, "→", pred, "| rama:", info["rama_predicha_nodo1"])


data/pruebas/e.jpg → clavo | rama: palo
data/pruebas/f.jpg → tuerca | rama: aro
data/pruebas/g.jpg → tuerca | rama: aro
data/pruebas/h.jpg → tuerca | rama: aro


* IDENTIFICAR COMANDO DE VOZ // KNN

In [15]:
from src.predecir_audio import predict_command

INPUT_WAV= "data_voz/pruebas/c.wav"  # AUDIO A PROCESAR
MODEL_PATH= "data_voz/models/knn_model.pkl"
SCALER_PATH        = "data_voz/models/scaler.pkl"
FEATURE_NAMES_PATH = "data_voz/models/feature_names.pkl"

predict_command(
    input_wav=INPUT_WAV,
    model_path=MODEL_PATH,
    scaler_path=SCALER_PATH,
    feature_names_path=FEATURE_NAMES_PATH,
    tmp_preprocessed="data_voz/tmp_preprocessed.wav",
    sample_rate=16000,
    n_mfcc=13
    )


[OK] Preprocesado: data_voz/pruebas/c.wav -> data_voz/tmp_preprocessed.wav
Archivo: data_voz/pruebas/c.wav
Predicción: contar
Probabilidades por clase:
  contar: 1.0000
  proporcion: 0.0000
  salir: 0.0000


/home/gabriel/Documents/IA/clasificador_piezas_ia_2025/.venv/lib/python3.12/site-packages/sklearn/utils/validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
